In [2]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time

In [5]:
consumer_key = 'pYoPARudnBJeg6VOos87eRL1I'
consumer_secret_key = '8fubRmzWeS8Iaor9KHDfgMivDEMsjGKPf64nZre9EnV2IWHnvU'
access_key = 'EMuVRHgtJj0yby1cPFgOavtdSJJQwfkdZOcn4IxjShfyQ'
access_key_secret = 'EMuVRHgtJj0yby1cPFgOavtdSJJQwfkdZOcn4IxjShfyQ'

auth = OAuthHandler(consumer_key, consumer_secret_key)
auth.set_access_token(access_key, access_key_secret)
api = tweepy.API(auth)


In [6]:
def scrape_tweets(search_words, date_since, num_tweets, num_runs):

    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times

    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags'])
    program_start = time.time()
    for i in range(0, num_runs):
        start_run = time.time()

        tweets = tweepy.Cursor(api.search, q=search_words, lang="en",
                               since=date_since, tweet_mode='extended').items(num_tweets)

        tweet_list = [tweet for tweet in tweets]

        no_tweets = 0

    for tweet in tweet_list:
# Pull the values
        username = tweet.user.screen_name
        acct_desc = tweet.user.description
        location = tweet.user.location
        following = tweet.user.friends_count
        followers = tweet.user.followers_count
        total_tweets = tweet.user.statuses_count
        user_created_ts = tweet.user.created_at
        tweet_created_ts = tweet.created_at
        retweet_count = tweet.retweet_count
        hashtags = tweet.entities['hashtags']
    try:
        text = tweet.retweeted_status.full_text
    except AttributeError:  # Not a Retweet
        text = tweet.full_text
# Add the 11 variables to the empty list - ith_tweet:
        ith_tweet = [username, acct_desc, location, following, followers, total_tweets,
                         user_created_ts, tweet_created_ts, retweet_count, text, hashtags]
# Append to dataframe - db_tweets
        db_tweets.loc[len(db_tweets)] = ith_tweet
# increase counter - noTweets
        no_tweets += 1

        # Run ended:
        end_run = time.time()
        duration_run = round((end_run-start_run)/60, 2)

        print('no. of tweets scraped for run {} is {}'.format(i + 1, no_tweets))
        print('time take for {} run to complete is {} mins'.format(i+1, duration_run))

        time.sleep(920) #15 minute sleep time
# Once all runs have completed, save them to a single csv file:
    from datetime import datetime

    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
# Define working path and filename
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_sahkprotests_tweets.csv'
# Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index=False)

    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [7]:
search_words = "#hongkong OR #hkprotests OR #freehongkong OR #hongkongprotests OR #hkpolicebrutality OR #antichinazi OR #standwithhongkong OR #hkpolicestate OR #HKpoliceterrorist OR #standwithhk OR #hkpoliceterrorism"
date_since = "2019-11-03"
num_tweets = 300
num_runs = 1
# Call the function scrape_tweets
scrape_tweets(search_words, date_since, num_tweets, num_runs)

TweepError: Twitter error response: status code = 401